In [ ]:
!cd ..
!pwd

/content


In [ ]:
%%capture
!rm -r GETAM
!git clone https://github.com/Hammad-Mir/GETAM

In [ ]:
%%capture
! pip install timm

In [ ]:
%%capture
!pip install git+https://github.com/lucasb-eyer/pydensecrf.git

In [ ]:
! pwd

/content


In [ ]:
import os
import cv2
import tarfile
import numpy as np
import pandas as pd
from math import ceil
from time import time
from tqdm.auto import tqdm
import multiprocessing as mp
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import timm
import torch
import torchvision
import torchsummary
from torch import nn
from torch.optim import Adam
import torch.nn.functional as F
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
#from torch.utils.tensorboard import SummaryWriter



%matplotlib inline

In [ ]:
os.chdir(f'./GETAM')

In [ ]:
import myTool as mytool
#from DenseEnergyLoss import DenseEnergyLoss
from DPT.DPT import DPT, DPTSegmentationModel
from tool import pyutils, imutils, torchutils
from myTool import compute_joint_loss, compute_seg_label_3, validation

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

# Setting up GPU

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
device

'cuda:0'

# Test images


In [ ]:
test_image_path_1  = '/content/drive/MyDrive/Newcastle_University/Final_Project_Dissertation/Viz_Test_Images/436219-7159-48057-[1, 0, 0].png'
#test_image_path_2  = '/content/drive/MyDrive/Newcastle_University/Final_Project_Dissertation/Viz_Test_Images/439448-38035-50808-[0, 1, 0].png'
valid_image_path   = '/content/drive/MyDrive/Newcastle_University/Final_Project_Dissertation/Viz_Test_Images/00.png'
valid_mask_path    = '/content/drive/MyDrive/Newcastle_University/Final_Project_Dissertation/Viz_Test_Images/00_1.png'

In [ ]:
test_image_1  = cv2.resize(cv2.imread(test_image_path_1), (224, 224))
#test_image_2  = cv2.resize(cv2.imread(test_image_path_2), (224, 224))
valid_image = cv2.resize(cv2.imread(valid_image_path), (224, 224))
valid_mask  = cv2.resize(cv2.imread(valid_mask_path), (224, 224))

In [ ]:
fig = plt.figure(figsize=(10, 10))

ax = fig.add_subplot(1, 3, 1)
plt.imshow(test_image_1)

ax = fig.add_subplot(1, 3, 2)
plt.imshow(valid_image)

ax = fig.add_subplot(1, 3, 3)
plt.imshow(cv2.cvtColor(valid_mask, cv2.COLOR_BGR2RGB))

In [ ]:
def preprocess_image(img: np.ndarray, mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], resized = (224, 224)) -> torch.Tensor:
  
  preprocessing = transforms.Compose([
                                      transforms.ToTensor(),
                                      transforms.Resize((224, 224)),
                                      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                      ])
  return preprocessing(img.copy()).unsqueeze(0)

In [ ]:
def reshape_transform(tensor, height=14, width=14):
  result = tensor[:, 1 :  , :].reshape(tensor.size(0),
                                       height, width, tensor.size(2))
  
  # Bring the channels to the first dimension,
  # like in CNNs.
  result = result.transpose(2, 3).transpose(1, 2)
  return result

In [ ]:
def show_mask_on_image(img, mask):
    img = np.float32(img) / 255
    mask = cv2.resize(mask, (224, 224))
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    return np.uint8(255 * cam)

# Creating the Model

In [ ]:
GETAM_model = DPTSegmentationModel(num_classes=3, backbone_name='vitb16_224').to(device)
GETAM_model.load_state_dict(torch.load(f='/content/drive/MyDrive/Newcastle_University/Final_Project_Dissertation/Models/border_cutmix_GETAM.pth',
                                       map_location=torch.device(device)))

cur_backbone: vitb16_224


In [ ]:
GETAM_model

In [ ]:
torch.summary(GETAM_model, (3, 224, 224))

# Generating Image CAMs